<a href="https://colab.research.google.com/github/GergelyTuri/sleep/blob/master/tsclustering2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
library(tidyverse)
install.packages("igraph")
library(googledrive)
library(readr)
library(igraph)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in install.packages("igraph"):
“installation of package ‘igraph’ had non-zero exit status”


ERROR: Error in library(igraph): there is no package called ‘igraph’


In [6]:
drive_auth()

Is it OK to cache OAuth access credentials in the folder ~/.cache/gargle
between R sessions?
1: Yes
2: No


Selection: 1


Please point your browser to the following url: 

https://accounts.google.com/o/oauth2/v2/auth?client_id=603366585132-frjlouoa3s2ono25d2l9ukvhlsrlnr7k.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email&redirect_uri=https%3A%2F%2Fwww.tidyverse.org%2Fgoogle-callback%2F&response_type=code&state=6db91495d660b022ebdbfda7d4a5a787&access_type=offline&prompt=consent



Enter authorization code: eyJjb2RlIjoiNC8wQWVhWVNIQzFGWHNKa1diMnFzYjR5eEF4OW16T3BZZ2xLbXdaQUx1ZDdpalhSNWlSa1FsMl9TZnJ6eHgwZ05GZHBrNHQwdyIsInN0YXRlIjoiNmRiOTE0OTVkNjYwYjAyMmViZGJmZGE3ZDRhNWE3ODcifQ==


In [8]:
file <- drive_ls(q = "/gdrive/Shareddrives/Turi_lab/Data/Sleep/2p")


ERROR: [1m[33mError[39m in `do_paginated_request()`:[22m
[1m[22m[33m![39m Client error: (403) Forbidden
Request had insufficient authentication scopes.
PERMISSION_DENIED
[36m•[39m message: Insufficient Permission
[36m•[39m domain: global
[36m•[39m reason: insufficientPermissions


In [9]:
ts_dist_awake = read.csv("/gdrive/Shareddrives/Turi_lab/Data/Sleep/2p/ts_dist_awake.csv")
ts_dist_sleep = read.csv('/gdrive/Shareddrives/Turi_lab/Data/Sleep/ts_dist_sleep.csv')

net.epsilon.create <- function(dist, epsilon) {
    n = matrix(0, ncol(dist), nrow(dist))
    n[dist < epsilon] = 1;
    return(graph.adjacency(n, mode="undirected", diag=F));
}

Warning message in file(file, "rt"):
“cannot open file '/gdrive/Shareddrives/Turi_lab/Data/Sleep/2p/ts_dist_awake.csv': No such file or directory”


ERROR: Error in file(file, "rt"): cannot open the connection


In [ ]:
eps_seq = seq(0.01,0.99,0.01)
ddd = function(ts_dist){
modu = c()
n_hubs = c()
n_edges = c()
for (i in 1:length(eps_seq)){
net1 = net.epsilon.create(dist = ts_dist, epsilon = eps_seq[i])
c1 = cluster_fast_greedy(net1)
hub_score1 = hub.score(net1)
n_hubs[i] = length(which(hub_score1[["vector"]]>0.9))
modu[i] = modularity(c1)
n_edges[i] = gsize(net1)
}
return(list(n_hubs,n_edges,modu))
}

tt = tibble(
  eps = rep(eps_seq,2),
  state = c(rep("aw",length(eps_seq)),rep("sl",length(eps_seq))),
  n_hub = c(ddd(ts_dist_awake)[[1]],ddd(ts_dist_sleep)[[1]]),
  n_edge = c(ddd(ts_dist_awake)[[2]],ddd(ts_dist_sleep)[[2]]),
  modu = c(ddd(ts_dist_awake)[[3]],ddd(ts_dist_sleep)[[3]])
)

tt %>%
  ggplot(aes(x=eps,y=modu,colour=state))+
  geom_point(size=0.8)+
  geom_line()

tt %>%
  ggplot(aes(x=eps,y=n_hub,colour=state))+
  geom_point(size=0.8)+
  geom_line()

tt %>%
  ggplot(aes(x=eps,y=n_edge,colour=state))+
  geom_point(size=0.8)+
  geom_line()

tt %>%
  ggplot(aes(x=n_edge,y=n_hub,colour=state))+
  geom_point(size=0.8)+
  geom_line()

## significant sleep Gergo
tibble(
  n = 1:length(rownames(roi_cluster)),
  roi = rownames(roi_cluster)) %>%
  filter(roi %in% sig_cell) %>% view

In [ ]:
ts_dist = ts_dist_sleep
eps = 0.2
net = net.epsilon.create(dist = ts_dist, epsilon = eps)
c = cluster_fast_greedy(net)
membership(c)
plot(net, vertex.color=membership(c), layout=layout_with_fr(net))
#hub score
hub_score <- hub.score(net)
which(hub_score[["vector"]]>=0.9)